# Segmenting and Clustering Neighbourhoods in Toronto

## PART ONE: SCRAPPING AND CLEANING THE TORONTO DATA ON WIKIPEDIA

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

print('Libraries imported.')

Libraries imported.


In [4]:
from bs4 import BeautifulSoup
link = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(link,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XjaFqgpAIC8AAL5QNVEAAABD","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":935851093,"wgRevisionId":935851093,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communi

In [5]:
table = soup.find('table',{'class':'wikitable sortable'})
print(table.tr.text)


Postcode
Borough
Neighbourhood



In [6]:
tab = table.find_all('tr')
my_table= ""
for tr in tab:
    row = ""
    for tds in tr.find_all('td'):
        row = row + "," + tds.text
    my_table= my_table + row[1:]
print(my_table)

M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Downtown Toronto,Queen's Park
M8A,Not assigned,Not assigned
M9A,Queen's Park,Not assigned
M1B,Scarborough,Rouge
M1B,Scarborough,Malvern
M2B,Not assigned,Not assigned
M3B,North York,Don Mills North
M4B,East York,Woodbine Gardens
M4B,East York,Parkview Hill
M5B,Downtown Toronto,Ryerson
M5B,Downtown Toronto,Garden District
M6B,North York,Glencairn
M7B,Not assigned,Not assigned
M8B,Not assigned,Not assigned
M9B,Etobicoke,Cloverdale
M9B,Etobicoke,Islington
M9B,Etobicoke,Martin Grove
M9B,Etobicoke,Princess Gardens
M9B,Etobicoke,West Deane Park
M1C,Scarborough,Highland Creek
M1C,Scarborough,Rouge Hill
M1C,Scarborough,Port Union
M2C,Not assigned,Not assigned
M3C,North York,Flemingdon Park
M3C,North York,Don Mills South
M4C,East York,Woodbine Heights
M5C,Downtown Toronto,St. James 

In [9]:
with open('toronto.csv', 'wb') as tor:
    tor.write(bytes(my_table, encoding='ascii',errors='ignore'))

In [10]:
tor_data = pd.read_csv("toronto.csv")
tor_data.columns= ["Postalcode","Borough","Neighbourhood"]
tor_data.head()

,Postalcode,Borough,Neighbourhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Heights


In [16]:
tor_data.shape

(286, 3)

In [17]:
tor_data.drop(tor_data[tor_data['Borough']=="Not assigned"].index, inplace=True)
tor_data.head()

,Postalcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor


In [27]:
tor_data.loc[tor_data['Neighbourhood'] =='Not assigned','Neighbourhood']=tor_data['Borough']
tor_data.sample(20)

,Postalcode,Borough,Neighbourhood
23,M9B,Etobicoke,Princess Gardens
92,M2K,North York,Bayview Village
243,M9W,Etobicoke,Northwest
195,M4T,Central Toronto,Summerhill East
33,M6C,York,Humewood-Cedarvale
241,M8W,Etobicoke,Alderwood
37,M9C,Etobicoke,Eringate
175,M9R,Etobicoke,Martin Grove Gardens
114,M5L,Downtown Toronto,Victoria Hotel
157,M6P,West Toronto,High Park


In [33]:
tor_df = tor_data.groupby(['Postalcode','Borough'],sort=False).agg( ','.join).reset_index()
tor_df.head(11)

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [34]:
tor_df.shape

(103, 3)

# PART TW0: OBTAINING THE GEOGRAPHICAL COORDINATES OF THE NEIGHBOURHOODS IN TORONTO

In [35]:
tor_geo = pd.read_csv('Geospatial_Coordinates.csv')
tor_geo.head(11)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [37]:
tor_geo.shape

(103, 3)

In [38]:
tor_df.rename(columns={'Postalcode':'Postal Code'}, inplace=True)
tor_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [39]:
tor_df2 = pd.merge(tor_df,tor_geo[['Postal Code','Latitude','Longitude']],on='Postal Code')
tor_df2.head(11)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [40]:
tor_df2.shape

(103, 5)

## PART 3: EXPLORING AND CLUSTERING THE NEIGHBOURHOODS IN TORONTO

### A. EXPLORATORY ANALYSIS OF NEIGHBOURHOODS

In [42]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent = 'Toronto')
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print("The geograpical coordinate of Toronto are {}, {}.".format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [44]:
my_toronto_map = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10 )

for lat, lng, borough, Neighbourhood in zip(tor_df2['Latitude'], tor_df2['Longitude'], 
                                            tor_df2['Borough'], tor_df2['Neighbourhood']):
    label = '{},{}'.format(Neighbourhood, borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(my_toronto_map)
    
my_toronto_map

In [54]:
from IPython.core.display import HTML
HTML(my_toronto_map._repr_html_())

In [45]:
CLIENT_ID = 'KUYU5TIHQBZSMBG25XCU3LSOQYUERZSGMG4SEXZWRQMGWWCN' 
CLIENT_SECRET = '3H21KUGZVEHRCFQ5OXZ5435Q1MLGFVYIEPM010GHE5RVDTAV' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KUYU5TIHQBZSMBG25XCU3LSOQYUERZSGMG4SEXZWRQMGWWCN
CLIENT_SECRET:3H21KUGZVEHRCFQ5OXZ5435Q1MLGFVYIEPM010GHE5RVDTAV


In [46]:
toronto_data = tor_df2[tor_df2['Borough'] == 'Toronto'].reset_index(drop=True)
toronto_data.shape

(0, 5)

In [48]:
tor_df_lat = tor_df2.loc[0, 'Latitude'] 
tor_df_long = tor_df2.loc[0, 'Longitude']

tor_df_name = tor_df2.loc[0, 'Neighbourhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(tor_df_name, 
                                                               tor_df_lat, 
                                                               tor_df_long))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [51]:
LIMIT=100
radius=500

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
          
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
tor_venues = getNearbyVenues(names=tor_df2['Neighbourhood'],latitudes=tor_df2['Latitude'],longitudes=tor_df2['Longitude'])


Parkwoods
Victoria Village
Harbourfront
Lawrence Heights,Lawrence Manor
Queen's Park
Queen's Park
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
The Danforth West,Riverdale
Design Exchange,Toronto 

In [58]:
print(tor_venues.shape)
tor_venues.head()

(2228, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [59]:
tor_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",10,10,10,10,10,10
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",25,25,25,25,25,25
Berczy Park,54,54,54,54,54,54


In [60]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique()tor)))

There are 264 uniques categories.


In [61]:
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

tor_onehot['Neighbourhood'] = tor_venues['Neighbourhood'] 

fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [62]:
tor_grouped = tor_onehot.groupby('Neighbourhood').mean().reset_index()
tor_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,0.000000,0.00,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.040000,0.000000,

In [64]:
tor_grouped.shape

(100, 265)

In [65]:
num_top_venues = 5

for hood in tor_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
          venue  freq
0   Coffee Shop  0.07
1          Café  0.04
2    Steakhouse  0.04
3           Bar  0.04
4  Burger Joint  0.03


----Agincourt----
                       venue  freq
0                     Lounge   0.2
1  Latin American Restaurant   0.2
2             Breakfast Spot   0.2
3             Clothing Store   0.2
4               Skating Rink   0.2


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                      venue  freq
0                      Park   0.5
1                Playground   0.5
2  Mediterranean Restaurant   0.0
3         Mobile Phone Shop   0.0
4        Miscellaneous Shop   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store   0.2
1  Fried Chicken Joint   0.1
2             Pharmacy   0.1
3          Pizza Place   0.1
4          Coffee Shop   0.1


----Alderwood,Long Branch----
            venue  f

            venue  freq
0   Grocery Store  0.25
1           Hotel  0.25
2            Bank  0.25
3            Park  0.25
4  Hardware Store  0.00


----Downsview,North Park,Upwood Park----
                        venue  freq
0                        Park  0.25
1                      Bakery  0.25
2  Construction & Landscaping  0.25
3            Basketball Court  0.25
4               Metro Station  0.00


----East Birchmount Park,Ionview,Kennedy Park----
                venue  freq
0      Discount Store  0.29
1   Convenience Store  0.14
2  Chinese Restaurant  0.14
3         Coffee Shop  0.14
4          Hobby Shop  0.14


----East Toronto----
               venue  freq
0               Park   0.2
1      Metro Station   0.2
2       Intersection   0.2
3  Convenience Store   0.2
4        Coffee Shop   0.2


----Emery,Humberlea----
                             venue  freq
0                   Baseball Field   1.0
1                Accessories Store   0.0
2  Molecular Gastronomy Restaurant   0.0
3 

4        Supermarket  0.06


----Victoria Village----
                   venue  freq
0  Portuguese Restaurant   0.2
1            Coffee Shop   0.2
2            Pizza Place   0.2
3           Intersection   0.2
4           Hockey Arena   0.2


----Westmount----
                venue  freq
0  Chinese Restaurant  0.17
1         Pizza Place  0.17
2         Coffee Shop  0.17
3      Sandwich Place  0.17
4      Discount Store  0.17


----Weston----
                        venue  freq
0           Convenience Store   1.0
1           Accessories Store   0.0
2  Modern European Restaurant   0.0
3           Mobile Phone Shop   0.0
4          Miscellaneous Shop   0.0


----Willowdale South----
                 venue  freq
0     Ramen Restaurant  0.09
1          Pizza Place  0.06
2       Sandwich Place  0.06
3  Japanese Restaurant  0.06
4                 Café  0.06


----Willowdale West----
            venue  freq
0        Pharmacy  0.17
1     Pizza Place  0.17
2     Coffee Shop  0.17
3  Discount Stor

In [66]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

tor_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
tor_neighborhoods_venues_sorted['Neighbourhood'] = tor_grouped['Neighbourhood']

for ind in np.arange(tor_grouped.shape[0]):
    tor_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

tor_neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Bar,Steakhouse,Café,Cosmetics Shop,Bakery,Burger Joint,Asian Restaurant,Thai Restaurant,Restaurant
1,Agincourt,Latin American Restaurant,Clothing Store,Lounge,Skating Rink,Breakfast Spot,Dumpling Restaurant,Eastern European Restaurant,Drugstore,Deli / Bodega,Donut Shop
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Coffee Shop,Pizza Place,Beer Store,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Liquor Store,Pharmacy,Comfort Food Restaurant
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Skating Rink,Gym,Pub,Pharmacy,Sandwich Place,Pool,Dessert Shop,Dance Studio


### B. CLUSTERING 

#### Using K-means for clustering neighbourhoods

In [83]:
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

kmeans.labels_[0:5] 

array([0, 0, 1, 0, 0], dtype=int32)

In [132]:
tor_neighborhoods_venues_sorted.drop(columns= ['Cluster Labels'], inplace = True)


In [133]:
tor_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
tor_merged = tor_df2

tor_merged = tor_merged.join(tor_neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

tor_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Discount Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Intersection,Hockey Arena,Pizza Place,Portuguese Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Bakery,Café,Pub,Park,Breakfast Spot,Restaurant,Mexican Restaurant,Spa,Hotel
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Accessories Store,Coffee Shop,Boutique,Vietnamese Restaurant,Event Space,Sporting Goods Shop,Miscellaneous Shop,Diner
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Park,Gym,Yoga Studio,Diner,Bar,Café,Portuguese Restaurant,Fast Food Restaurant,Beer Bar


In [134]:
tor_merged.isna().sum()

Postal Code               0
Borough                   0
Neighbourhood             0
Latitude                  0
Longitude                 0
Cluster Labels            2
1st Most Common Venue     2
2nd Most Common Venue     2
3rd Most Common Venue     2
4th Most Common Venue     2
5th Most Common Venue     2
6th Most Common Venue     2
7th Most Common Venue     2
8th Most Common Venue     2
9th Most Common Venue     2
10th Most Common Venue    2
dtype: int64

In [135]:
tor_merged.dropna(inplace=True)
tor_merged.isna().sum()

Postal Code               0
Borough                   0
Neighbourhood             0
Latitude                  0
Longitude                 0
Cluster Labels            0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

In [136]:
tor_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 102
Data columns (total 16 columns):
Postal Code               101 non-null object
Borough                   101 non-null object
Neighbourhood             101 non-null object
Latitude                  101 non-null float64
Longitude                 101 non-null float64
Cluster Labels            101 non-null float64
1st Most Common Venue     101 non-null object
2nd Most Common Venue     101 non-null object
3rd Most Common Venue     101 non-null object
4th Most Common Venue     101 non-null object
5th Most Common Venue     101 non-null object
6th Most Common Venue     101 non-null object
7th Most Common Venue     101 non-null object
8th Most Common Venue     101 non-null object
9th Most Common Venue     101 non-null object
10th Most Common Venue    101 non-null object
dtypes: float64(3), object(13)
memory usage: 13.4+ KB


In [138]:
tor_merged = tor_merged.astype({"Cluster Labels": int})
tor_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 102
Data columns (total 16 columns):
Postal Code               101 non-null object
Borough                   101 non-null object
Neighbourhood             101 non-null object
Latitude                  101 non-null float64
Longitude                 101 non-null float64
Cluster Labels            101 non-null int64
1st Most Common Venue     101 non-null object
2nd Most Common Venue     101 non-null object
3rd Most Common Venue     101 non-null object
4th Most Common Venue     101 non-null object
5th Most Common Venue     101 non-null object
6th Most Common Venue     101 non-null object
7th Most Common Venue     101 non-null object
8th Most Common Venue     101 non-null object
9th Most Common Venue     101 non-null object
10th Most Common Venue    101 non-null object
dtypes: float64(2), int64(1), object(13)
memory usage: 13.4+ KB


In [139]:
tor_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Park,Food & Drink Shop,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Discount Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Intersection,Hockey Arena,Pizza Place,Portuguese Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Bakery,Café,Pub,Park,Breakfast Spot,Restaurant,Mexican Restaurant,Spa,Hotel
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0,Furniture / Home Store,Clothing Store,Accessories Store,Coffee Shop,Boutique,Vietnamese Restaurant,Event Space,Sporting Goods Shop,Miscellaneous Shop,Diner
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Park,Gym,Yoga Studio,Diner,Bar,Café,Portuguese Restaurant,Fast Food Restaurant,Beer Bar


In [140]:
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighbourhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [141]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Coffee Shop,Intersection,Hockey Arena,Pizza Place,Portuguese Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,Downtown Toronto,0,Coffee Shop,Bakery,Café,Pub,Park,Breakfast Spot,Restaurant,Mexican Restaurant,Spa,Hotel
3,North York,0,Furniture / Home Store,Clothing Store,Accessories Store,Coffee Shop,Boutique,Vietnamese Restaurant,Event Space,Sporting Goods Shop,Miscellaneous Shop,Diner
4,Downtown Toronto,0,Coffee Shop,Park,Gym,Yoga Studio,Diner,Bar,Café,Portuguese Restaurant,Fast Food Restaurant,Beer Bar
5,Queen's Park,0,Coffee Shop,Park,Gym,Yoga Studio,Diner,Bar,Café,Portuguese Restaurant,Fast Food Restaurant,Beer Bar
6,Scarborough,0,Fast Food Restaurant,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
7,North York,0,Basketball Court,Café,Gym / Fitness Center,Baseball Field,Japanese Restaurant,Caribbean Restaurant,Yoga Studio,Diner,Discount Store,Dog Run
8,East York,0,Pizza Place,Fast Food Restaurant,Athletics & Sports,Breakfast Spot,Gym / Fitness Center,Gastropub,Pet Store,Pharmacy,Bank,Bus Line
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Tea Room,Restaurant,Bookstore,Ramen Restaurant,Bakery
10,North York,0,Park,Pub,Asian Restaurant,Pizza Place,Japanese Restaurant,Yoga Studio,Diner,Deli / Bodega,Department Store,Dessert Shop


In [142]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Park,Food & Drink Shop,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Discount Store
21,York,1,Park,Market,Fast Food Restaurant,Women's Store,Airport Food Court,College Gym,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
40,North York,1,Park,Airport,Bus Stop,Snack Place,College Rec Center,College Stadium,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
46,North York,1,Park,Grocery Store,Bank,Hotel,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
49,North York,1,Park,Construction & Landscaping,Bakery,Basketball Court,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Deli / Bodega,Eastern European Restaurant
61,Central Toronto,1,Park,Bus Line,Swim School,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Ethiopian Restaurant
66,North York,1,Park,Bank,Bar,Convenience Store,Discount Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Deli / Bodega
68,Central Toronto,1,Trail,Park,Jewelry Store,Sushi Restaurant,Yoga Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
83,Central Toronto,1,Park,Gym,Tennis Court,Playground,Yoga Studio,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
85,Scarborough,1,Park,Playground,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run


In [143]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,2,Jewelry Store,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


In [144]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,3,Playground,Yoga Studio,Dog Run,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant


In [145]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,4,Baseball Field,Yoga Studio,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
101,Etobicoke,4,Locksmith,Baseball Field,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant


In [146]:
tor_merged.loc[tor_merged['Cluster Labels'] == 5, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
